# Design of Experiments

This notebook accompanies the ECE595 Data analytics course taught at Purdue in Fall 2022. These set of examples pertain to the materials of [Lecture 8.](https://github.com/alam740/Data-Analytics-Course/blob/master/Lecture-PDFs/ECE%20595%20-%20Lecture%2008.pdf)

*Written by Jabir Bin Jahangir (jabir@purdue.edu)*

# Experiment on a black box

The `blackbox` function defined below represents a black box under a 7-factor, 2-level experiment. Given a test matrix the function returns the result of the experiment. Each row of the test matrix defines the , the levels (either 1 or 2) of every factor for a single run of the experiment.  

Run the following cell to initiate the blackbox.

In [ ]:
#@title
import math 
import numpy as np
from numpy import matlib

def blackbox(levels):
  x1 = np.transpose(levels); 

  x1_step1_xoffset = np.array([[1,1,1,1,1,1,1]]).transpose();
  x1_step1_gain = np.array([[2,2,2,2,2,2,2]]).transpose();
  x1_step1_ymin = -1;

  b1 = np.array([[1.7237835335687619054], [2.5335770808130533283]]);
  IW1_1 = np.array([[-0.13131532050441799275, -1.5767097360939066331, -2.0257653566958220281, 2.3756010526864961285, 0.50913244334160911997, -0.56307817418422101419, -0.45473557435243810998],
          [-1.114453933075197245, -0.036113288529629979096, -0.057911951730746119571, -0.45888814269827166159, -1.2076271133400777735, 1.4126122113674441927, 1.8924261635012584737]]);

  b2 = -0.71748175286842708065;
  LW2_1 = np.array([[0.80559331477697015966, -0.9136975900783510518]]);

  # Output 1
  y1_step1_ymin = -1;
  y1_step1_gain = 0.2;
  y1_step1_xoffset = 9;

  # mapminmax 
  Q  = levels.shape[0]
  a1 = x1 - x1_step1_xoffset
  a2 = np.multiply(a1, x1_step1_gain)
  xp1 = a2 + x1_step1_ymin; 

  # Layer 1 
  wxp1 = np.matmul(IW1_1, xp1)
  l1 = np.matlib.repmat(b1, 1, Q) + wxp1

  def tsig(t):
    return 2 / (1 + np.exp(-2*t)) - 1; 
  np.vectorize(tsig)
  l1_o = tsig(l1)

  #Layer 2
  out = np.matlib.repmat(b2, 1, Q) + np.matmul(LW2_1, l1_o)

  # output reverse mapminmax
  b1 = out - y1_step1_ymin; 
  b2 = np.divide(b1, y1_step1_gain); 
  return  b2 + y1_step1_xoffset;


In [ ]:
# Test matrix for 8 runs

levels = np.array([[1,1,	1,	1,	1,	1,	1],
[2,	1,	1,	1	,1	,1,	1],
[2,	2,	1,	1,	1,	1,	1],
[2,	1,	2,	1,	1,	1,	1],
[2,	1,	1,	2,	1,	1,	1],
[2,	1,	1,	2,	2,	1,	1],
[2,	1,	1,	2,	2,	2,	1],
[2,	1,	1,	2,	2,	1	,2]]); 

result = blackbox(levels)
print(result); 

[[10.         15.         12.          9.         17.99999997 19.00000001
  17.         13.        ]]


# Full factorial design

A full factorial design will entail $2^7 = 127$ runs. 

# Fractional factorial designs

Using fractional design strategies, we can optimally choose a subset of all possible combinations. 


In [ ]:
!pip install pyDOE2 
from pyDOE2 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Example 1: 2-level, 4-factor fractional design

Generate a fractional factorial design of four factors/variables each with 2 levels ($\pm 1$). The first three variables (a, b, c) are basic factors  and the fourth (abc) is generated by the product of the three basic factors representing interactions.  

In [ ]:
gen = 'a b c abc'

d1 = fracfact(gen)
print(d1)

d1.shape

[[-1. -1. -1. -1.]
 [ 1. -1. -1.  1.]
 [-1.  1. -1.  1.]
 [ 1.  1. -1. -1.]
 [-1. -1.  1.  1.]
 [ 1. -1.  1. -1.]
 [-1.  1.  1. -1.]
 [ 1.  1.  1.  1.]]


(8, 4)

## Example 2: 2-level, 6-factor fractional design

In [ ]:
gen2 = 'a b c d bcd acd';

d2 = fracfact(gen2)
print(d2)

d2.shape

[[-1. -1. -1. -1. -1. -1.]
 [ 1. -1. -1. -1. -1.  1.]
 [-1.  1. -1. -1.  1. -1.]
 [ 1.  1. -1. -1.  1.  1.]
 [-1. -1.  1. -1.  1.  1.]
 [ 1. -1.  1. -1.  1. -1.]
 [-1.  1.  1. -1. -1.  1.]
 [ 1.  1.  1. -1. -1. -1.]
 [-1. -1. -1.  1.  1.  1.]
 [ 1. -1. -1.  1.  1. -1.]
 [-1.  1. -1.  1. -1.  1.]
 [ 1.  1. -1.  1. -1. -1.]
 [-1. -1.  1.  1. -1. -1.]
 [ 1. -1.  1.  1. -1.  1.]
 [-1.  1.  1.  1.  1. -1.]
 [ 1.  1.  1.  1.  1.  1.]]


(16, 6)

# Taguchi Orthogonal Arrays

Taguchi orthogonal arrays have the following property: For each level of any chosen parameter, every level of all other parameters are tested at least once. The combinations of parameter level is also minimally repeated and each level for each factor appears in equal number of times. Test matrix represented by an orthogonal array leads to lower number of runs (viz. lower time and resource cost) to explore the parameter space compared to a naive full factorial design.

Taguchi test matrices: http://reliawiki.org/index.php/Taguchi_Orthogonal_Arrays


## Example 1: Experiment with Taguchi OA

In [ ]:
# Using L8 for the 2-level, 7-factor experiment
# 2 levels of parameters tested 4 times. 

taguchi_l8 = np.array([
    [1, 1,	1,	1,	1,	1,	1],
    [1,	1,	1,	2,  2,  2,	2],
    [1,	2,	2,	1,	1,	2,	2],
    [1,	2,	2,	2,	2,	1,	1],
    [2,	1,	2,	1,	2,	1,	2],
    [2,	1,	2,	2,	1,	2,	1],
    [2,	2,	1,	1,	2,	2,	1],
    [2,	2,	1,	2,	1,	1	, 2 ]]); 

result_Taguchi = blackbox(taguchi_l8)
print(result_Taguchi); 

[[10.          9.87226013  1.81623676 18.39558842  4.50490943 10.16280675
   9.72503199  9.91311133]]
